Wahlpflichtfach Künstliche Intelligenz II: Praktikum 

---

# 12 - Tensoflow Grundlagen

Nachdem wir uns gerade angeguckt haben, wie wir eigene Schichten erstellen und kombinieren, das neuronale Netz optimieren und den Trainingsfortschritt visualisieren, wollen wir uns jetzt mit den Built-In Funktionen von Tensorflow vertraut machen.

In [ ]:
import datetime

import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

In [ ]:
%matplotlib inline
%load_ext tensorboard

In [ ]:
print(tf.__version__)

## Erstellen eines kleinen Regressions-Datensatz
Zuerst erstell wir uns erneut einen Dummy-Regressions-Datensatz 

In [ ]:
xs = np.linspace(-5,5, 20, dtype=np.float32)
def f(x):
    return 0.005*(-x**3+6*x**2+2*x)+0.3
ys = f(xs)

training_data_ids = np.random.choice(20,15, replace=False)
test_data_ids = np.array([i if i not in training_data_ids else 99 for i in range(20)])
test_data_ids = test_data_ids[test_data_ids < 21]
training_data_xs = xs[training_data_ids]
training_data_ys = ys[training_data_ids]
test_data_xs = xs[test_data_ids]
test_data_ys = ys[test_data_ids]

plt.scatter(training_data_xs, training_data_ys, c='red')
plt.scatter(test_data_xs, test_data_ys, c='blue')
plt.legend(("Training Data","Test Data"), loc='upper right')
plt.xlim(-5,5)
plt.ylim(0.2,1.7)
plt.show()

Wir müssen nicht unbedingt `tf.data.Dataset`s erstellen. Aber wir müssen trotzdem die Shape der Arrays anpassen, so dass sie 2 Dimensionen haben. 

In [ ]:
x_train = np.expand_dims(training_data_xs, axis=1)
x_test = np.expand_dims(test_data_xs, axis=1)
y_train = np.expand_dims(training_data_ys, axis=1)
y_test = np.expand_dims(test_data_ys, axis=1)

## Erstellen des neuronalen Netzes
In der nächsten Zelle erstellen wir das neuronale Netz. Dafür verwenden wir ein [sequentielles Modell](https://www.tensorflow.org/api_docs/python/tf/keras/Sequential?hl=en). Diese hilft dabei ein Modell aus einer Liste von Schichten zu erstellen und die Schichten zu trainieren. Als Schichten nehmen wir das [Dense](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Dense?hl=en)-Layer.

In [ ]:
from keras import Sequential
from keras.layers import Dense

# clear the session
tf.keras.backend.clear_session()

# create the model
model = Sequential([Dense(512, activation='relu'), Dense(1)])
model.compile(optimizer='adam',
              loss='mse',
              metrics='mse')

## Einrichten von TensorBoard
Um die Visualisierung des Trainings zu automatisieren, kann [TensorBoard](https://www.tensorflow.org/tensorboard) verwendet werden. Dieses hilft dabei den Trainingsfortschritt zu visualisieren. Um TensorBoard im Training zu aktivieren müssen wir uns eine callback-Funktion erstellen ([Doku](https://www.tensorflow.org/api_docs/python/tf/keras/callbacks/TensorBoard?hl=en)). 

In [ ]:
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

## Trainieren des neuronalen Netzes
Für das Training müssen wir nicht unbedingt eine Trainingschleife schreiben. Stattdessen können wir auch die `fit()`-Methode des `Sequential`-Modells verwenden.

In [ ]:
model.fit(x=x_train, 
          y=y_train,
          batch_size=15,
          epochs=1000, 
          validation_data=(x_test, y_test), 
          callbacks=[tensorboard_callback])

## TensorBoard
Das TensorBoard kann ganz einfach innerhalb von Jupyter Notebook verwendet werden.

In [ ]:
%tensorboard --logdir logs

## Vorhersagen treffen
Um für neue Daten eine Vorhersage zu treffen, kann die `predict()`-Methode des `Sequential`-Modells verwendet werden.

In [ ]:
# Plot training and test data.
plt.figure()
plt.scatter(training_data_xs, training_data_ys, c='red')
plt.scatter(test_data_xs, test_data_ys, c='blue')

# Calculate the network's output for 100 xs.
xs = np.linspace(-5,5,100, dtype=np.float32)
xs = np.reshape(xs, newshape=(-1,1))
ys = model.predict(xs)

# Plot it.
plt.plot(xs,ys)
plt.xlim(-5,5)
plt.ylim(0.2,1.7)
plt.show()

---

Wahlpflichtach Künstliche Intelligenz II: Praktikum 